In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
gpt4_result_justification = pd.read_excel('transformed/gpt4_result_justification.xlsx')
gemini_result_justification = pd.read_excel('transformed/gemini_result_justification.xlsx')
gpt4_result_justification = pd.read_excel('transformed/gpt4_result_justification.xlsx')
gemini_result_justification = pd.read_excel('transformed/gemini_result_justification.xlsx')

gpt35_result_justification = pd.read_excel('transformed/gpt35_result_justification.xlsx')
gemini_old_result_justification = pd.read_excel('transformed/gemini_old_result_justification.xlsx')
gpt35_result_justification = pd.read_excel('transformed/gpt35_result_justification.xlsx')
gemini_old_result_justification = pd.read_excel('transformed/gemini_old_result_justification.xlsx')

In [4]:
gpt4_result_justification.head()

,type,type (EI),type (NS),type (FT),type (JP),1st choice,1st choice (EI),1st choice (NS),1st choice (FT),1st choice (JP),is_matched(full),is_matched(EI),is_matched(NS),is_matched(FT),is_matched(JP),posts_filtered,Justification for 1st choice (EI),Justification for 1st choice (NS),Justification for 1st choice (TF),Justification for 1st choice (PJ)
0,INFJ,I,N,F,J,ENTP,E,N,T,P,False,False,True,False,False,'1.) Is there something that you've always wan...,Displays a range of interests and adaptability...,Shows a preference for exploring abstract idea...,Focuses on logical analysis but also values em...,"Demonstrates flexibility and spontaneity, char..."
1,INFJ,I,N,F,J,INFP,I,N,F,P,False,True,True,True,False,"'I don't know what to do anymore. At dinner,...",Reflects introspection and a preference for so...,Emphasizes emotional depth and abstract expres...,Prioritizes personal values and emotional unde...,Shows a contemplative and open-ended approach ...
2,INFJ,I,N,F,J,ENTP,E,N,T,P,False,False,True,False,False,"'{html link} an {MBTI type}, accepting others ...","Engages in diverse discussions, suggesting ext...","Prefers theoretical and abstract discussions, ...","Analytical and objective in decision-making, a...","Adaptable and open to new ideas, traits of per..."
3,INFJ,I,N,F,J,ESTP,E,S,T,P,False,False,False,False,False,"He was an extrovert, you can see his extrovers...","Focuses on dynamic social interaction, indicat...","Emphasizes real-world experiences, suggesting ...","Analyzes characters and situations logically, ...","Shows flexibility and spontaneity, characteris..."
4,INFJ,I,N,F,J,ENFP,E,N,F,P,False,False,True,True,False,'Go in for the kiss. Or if you're too scared ...,Engages in expressive and communicative intera...,"Explores imaginative and conceptual ideas, ind...",Values emotional connections and empathetic un...,Demonstrates an open-ended and exploratory app...


# SANDBOX

In [8]:
mbti_descriptions = pd.read_excel('raw/MBTI_descriptions.xlsx')
mbti_descriptions = dict(zip(mbti_descriptions['type'], mbti_descriptions['description']))
mbti_descriptions

{'ISTJ': 'Quiet, serious, earn success by being thorough and dependable. Practical, matter-of-fact, realistic, and responsible. Decide logically what should be done and work toward it steadily, regardless of distractions. Take pleasure in making everything orderly and organized—their work, their home, their life. Value traditions and loyalty.  ',
 'ISFJ': 'Quiet, friendly, responsible, and conscientious. Committed and steady in meeting their obligations. Thorough, painstaking, and accurate. Loyal, considerate, notice and remember specifics about people who are important to them, concerned with how others feel. Strive to create an orderly and harmonious environment at work and at home.',
 'INFJ': 'Seek meaning and connection in ideas, relationships, and material possessions. Want to understand what motivates people and are insightful about others. Conscientious and committed to their firm values. Develop a clear vision about how best to serve the common good. Organized and decisive in i

In [9]:
# Text Similarity Analysis
def compute_similarity(text, type_description):
    vectorizer = TfidfVectorizer().fit_transform([text, type_description])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0, 1]

In [10]:
gpt4_result_justification['similarity_score'] = gpt4_result_justification.apply(lambda row: 
                                                                                compute_similarity(row['Justification for 1st choice (EI)'], 
                                                                                                   mbti_descriptions.get(row['type'], '')), 
                                                                                                   axis=1
                                                                                                   )
gpt4_result_justification

,type,type (EI),type (NS),type (FT),type (JP),1st choice,1st choice (EI),1st choice (NS),1st choice (FT),1st choice (JP),is_matched(full),is_matched(EI),is_matched(NS),is_matched(FT),is_matched(JP),posts_filtered,Justification for 1st choice (EI),Justification for 1st choice (NS),Justification for 1st choice (TF),Justification for 1st choice (PJ),similarity_score
0,INFJ,I,N,F,J,ENTP,E,N,T,P,False,False,True,False,False,'1.) Is there something that you've always wan...,Displays a range of interests and adaptability...,Shows a preference for exploring abstract idea...,Focuses on logical analysis but also values em...,"Demonstrates flexibility and spontaneity, char...",0.112322
1,INFJ,I,N,F,J,INFP,I,N,F,P,False,True,True,True,False,"'I don't know what to do anymore. At dinner,...",Reflects introspection and a preference for so...,Emphasizes emotional depth and abstract expres...,Prioritizes personal values and emotional unde...,Shows a contemplative and open-ended approach ...,0.105513
2,INFJ,I,N,F,J,ENTP,E,N,T,P,False,False,True,False,False,"'{html link} an {MBTI type}, accepting others ...","Engages in diverse discussions, suggesting ext...","Prefers theoretical and abstract discussions, ...","Analytical and objective in decision-making, a...","Adaptable and open to new ideas, traits of per...",0.048848
3,INFJ,I,N,F,J,ESTP,E,S,T,P,False,False,False,False,False,"He was an extrovert, you can see his extrovers...","Focuses on dynamic social interaction, indicat...","Emphasizes real-world experiences, suggesting ...","Analyzes characters and situations logically, ...","Shows flexibility and spontaneity, characteris...",0.000000
4,INFJ,I,N,F,J,ENFP,E,N,F,P,False,False,True,True,False,'Go in for the kiss. Or if you're too scared ...,Engages in expressive and communicative intera...,"Explores imaginative and conceptual ideas, ind...",Values emotional connections and empathetic un...,Demonstrates an open-ended and exploratory app...,0.165121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,ESFJ,E,S,F,J,ISFP,I,S,F,P,False,False,True,True,False,'Prince - I Would Die 4 U Sent from my iPhon...,The person's posts focus significantly on sens...,The emphasis on sensory experiences in their c...,The posts lack sufficient information to defin...,The content is more focused on current experie...,0.115463
620,ESFJ,E,S,F,J,ENFJ,E,N,F,J,False,True,False,True,True,'I'm not sure what's better... posting individ...,The person's communication style is open and e...,"The focus on personal relationships, social dy...",The strong emphasis on emotional connections a...,The person demonstrates a desire for harmony a...,0.202947
621,ESFJ,E,S,F,J,ESFJ,E,S,F,J,True,True,True,True,True,'I don't hang out in Personality Cafe much any...,The speaker's strong preference for social int...,The emphasis on personal experience and concre...,The clear preference for making decisions base...,Their organized approach to giving advice and ...,0.150354
622,ESFJ,E,S,F,J,ESFJ,E,S,F,J,True,True,True,True,True,Most millennials I know who smoke excessively ...,The content indicates a strong social orientat...,The detailed sharing of personal experiences a...,"The concern for harmony, understanding, and pe...",The structured advice and concern for order an...,0.075289


In [ ]:
# Sentiment Analysis
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    return sia.polarity_scores(text)['compound']

In [ ]:
gpt4_result_justification['sentiment_score'] = gpt4_result_justification['Justification for 1st choice (EI)'].apply(analyze_sentiment)

In [ ]:
# Keyword Frequency Analysis
def keyword_frequency(text, keywords):
    words = Counter(text.split())
    return sum(words[k] for k in keywords if k in words)

In [ ]:
# Example keywords for each MBTI dimension
ei_keywords = ['reflection', 'interaction', 'solitude']
ns_keywords = ['abstract', 'practical']

data['ei_keyword_count'] = data['Justification for 1st choice (EI)'].apply(lambda x: keyword_frequency(x, ei_keywords))

# Visualizing Keyword Frequency
plt.hist(data['ei_keyword_count'], bins=20)
plt.title("Keyword Frequency for EI")
plt.xlabel("Count")
plt.ylabel("Frequency")
plt.show()

# TASK1: TBD

# TASK2: TBD

# TASK3: TBD